In [1]:
import pandas as pd
from openai import OpenAI

from dotenv import load_dotenv
load_dotenv()
import os

In [2]:
openai_api_key = os.getenv("openai_api_key")

In [3]:
data = {
    "post": [
        "Цей продукт дуже хороший!",
        "Не рекомендую, поганий сервіс.",
        "Ціна нормальна, але доставка довга.",
        "Дорогувато, але якість на висоті.",
        "Дорого, богато",
        "Зламалось майже зразу але сервіс вийшов на звязок і ми це питання вирішили"
    ]
}
df = pd.DataFrame(data)
df

,post
0,Цей продукт дуже хороший!
1,"Не рекомендую, поганий сервіс."
2,"Ціна нормальна, але доставка довга."
3,"Дорогувато, але якість на висоті."
4,"Дорого, богато"
5,Зламалось майже зразу але сервіс вийшов на звя...


In [4]:
client = OpenAI(
    api_key=openai_api_key,
)


def label_posts_in_batches(df, batch_size=10):
    all_labels = []
    for i in range(0, len(df), batch_size):
        batch = df["post"].iloc[i:i + batch_size].tolist()
        prompt = f"""
        Оцініть наступні тексти і дайте їм категорії:
        1. Позитивний
        2. Нейтральний
        3. Негативний
        Тексти: {batch}
        Ваша відповідь повинна бути списком чисел (1, 2 або 3), де кожне число відповідає тексту у тому ж порядку.
        """
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Ви - помічник, який класифікує тексти."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1000,
            temperature=0
        )
        labels = eval(response.choices[0].message.content.strip())
        all_labels.extend(labels)
    return all_labels


labels = label_posts_in_batches(df, batch_size=10)
df["label"] = labels

df

,post,label
0,Цей продукт дуже хороший!,1
1,"Не рекомендую, поганий сервіс.",3
2,"Ціна нормальна, але доставка довга.",2
3,"Дорогувато, але якість на висоті.",1
4,"Дорого, богато",3
5,Зламалось майже зразу але сервіс вийшов на звя...,1
